In [1]:
#import libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#set the path
root = os.path.dirname(os.path.realpath(''))
data = os.path.join(root,'Data')
data

'D:\\Dortmund\\Case Study\\Data'

In [3]:
data_list = os.listdir(data)

In [6]:
from datetime import datetime

datetime.datetime(2025, 4, 16, 17, 35, 1, 305358)

In [16]:
import pandas as pd
import numpy as np
import os
import re
# Path to folder containing all 33 CSVs
data_folder = 'D:\\Dortmund\\Case Study\\Data'  # Update to your actual path
file_list = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

# To store results for each file
all_smoothed_dfs = {}

# Process each CSV file
for file in file_list:  # Update to process all files as needed
    print(f"Processing {file}...")
    
    # Read the file
    df = pd.read_csv(os.path.join(data_folder, file))
    df = df.rename(columns={'index': 'timestamp'})
    df = df[['timestamp', 'HAUSHALT_TOT']]

    # Step 1: Fix time format and set the index to timestamp
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)

    # Step 2: Create rolling averages per hour, including past and future 2 records
    result = []
    for date, group in df.groupby(df.index.date):
        row = {}
        
        for h in range(24):
            target_time = pd.Timestamp(f'{date} {h:02d}:00:00')
            
            # Create time slice including past 2 records and future 2 records
            time_slice = pd.date_range(start=target_time - pd.Timedelta(minutes=30),  # 2 previous records
                                       end=target_time + pd.Timedelta(minutes=30),    # 2 future records
                                       freq='15T')
            try:
                values = df.loc[time_slice, 'HAUSHALT_TOT']
                row[f'ya_{h}'] = values.mean()  # Calculate average of all 7 values (2 past + current + 2 future)
            except KeyError:
                row[f'ya_{h}'] = np.nan  # Handle missing values if there's a problem with time slice
            #values = df.loc[time_slice, 'HAUSHALT_TOT']
            #row[f'ya_{h}'] = values.mean()  # Calculate average of all 7 values (2 past + current + 2 future)
            #except KeyError:
            #    row[f'ya_{h}'] = np.nan  # Handle missing values if there's a problem with time slice
                
        result.append(row)
    
    # Final DataFrame after averaging
    final_df = pd.DataFrame(result)
    final_df.index = pd.date_range(start=df.index.min().date(), periods=len(final_df), freq='D')
    final_df.index.name = 'date'
    final_df.reset_index(inplace=True)

    # Step 3: Add weekday info
    final_df['date'] = pd.to_datetime(final_df['date'])
    final_df['weekday'] = final_df['date'].dt.weekday

    # Step 4: Smoothing using ±2 weeks of same weekday
    def get_smoothed_row(df, current_index):
        row = {}
        current_date = df.loc[current_index, 'date']
        current_weekday = df.loc[current_index, 'weekday']
        date_range = [
            current_date - pd.Timedelta(weeks=2),
            current_date - pd.Timedelta(weeks=1),
            current_date,
            current_date + pd.Timedelta(weeks=1),
            current_date + pd.Timedelta(weeks=2)
        ]
        matching_rows = df[df['date'].isin(date_range)]
        for h in range(24):
            col = f'ya_{h}'
            row[f'yb_{h}'] = matching_rows[col].mean() if col in matching_rows else np.nan
        return row

    smoothed_data = []
    for i in range(len(final_df)):
        smoothed_row = get_smoothed_row(final_df, i)
        smoothed_data.append(smoothed_row)

    smoothed_df = pd.DataFrame(smoothed_data)
    smoothed_df.index = final_df['date']

    household_id = re.findall(r'\d+', file)
    smoothed_df['household'] = int(household_id[0]) if household_id else None

    # Store the smoothed DataFrame in the dictionary
    all_smoothed_dfs[file.replace('.csv', '')] = smoothed_df

    # Optional: Save each smoothed DataFrame as a CSV (individual files)
    smoothed_df.to_csv(os.path.join(data_folder, f'smoothed_{file}'))

print("✅ All files processed.")

# 1. Combine all DataFrames into one with household column
combined_df = []

for name, df in all_smoothed_dfs.items():
    df = df.copy()
    df['household'] = name  # Add household identifier
    combined_df.append(df)

# 2. Concatenate all dataframes into one final DataFrame
final_combined_df = pd.concat(combined_df)
final_combined_df.reset_index(inplace=True)  # Reset the index to make 'date' a column

# 3. Merge `ya_*` and `yb_*` columns based on date
final_combined_df = final_combined_df.merge(final_df[['date'] + [f'ya_{h}' for h in range(24)]], 
                                            on='date', 
                                            how='left')

# 4. Save the combined DataFrame to a single CSV
final_combined_df.to_csv(os.path.join(data_folder, 'all_households_smoothed.csv'), index=False)

print("📁 Combined file saved as 'all_households_smoothed.csv'")

Processing SFH10.csv...


C:\Users\mi\AppData\Local\Temp\ipykernel_25472\2942837334.py:34: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_slice = pd.date_range(start=target_time - pd.Timedelta(minutes=30),  # 2 previous records


Processing SFH11.csv...


C:\Users\mi\AppData\Local\Temp\ipykernel_25472\2942837334.py:34: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_slice = pd.date_range(start=target_time - pd.Timedelta(minutes=30),  # 2 previous records


Processing SFH12.csv...


...

Processing SFH8.csv...


C:\Users\mi\AppData\Local\Temp\ipykernel_25472\2942837334.py:34: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_slice = pd.date_range(start=target_time - pd.Timedelta(minutes=30),  # 2 previous records


Processing SFH9.csv...


C:\Users\mi\AppData\Local\Temp\ipykernel_25472\2942837334.py:34: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_slice = pd.date_range(start=target_time - pd.Timedelta(minutes=30),  # 2 previous records


✅ All files processed.
📁 Combined file saved as 'all_households_smoothed.csv'


In [18]:
final_combined_df = final_combined_df[['date','ya_0', 'ya_1', 'ya_2', 'ya_3', 'ya_4', 'ya_5', 'ya_6',
       'ya_7', 'ya_8', 'ya_9', 'ya_10', 'ya_11', 'ya_12', 'ya_13', 'ya_14',
       'ya_15', 'ya_16', 'ya_17', 'ya_18', 'ya_19', 'ya_20', 'ya_21', 'ya_22',
       'ya_23','yb_0', 'yb_1', 'yb_2', 'yb_3', 'yb_4', 'yb_5', 'yb_6', 'yb_7',
       'yb_8', 'yb_9', 'yb_10', 'yb_11', 'yb_12', 'yb_13', 'yb_14', 'yb_15',
       'yb_16', 'yb_17', 'yb_18', 'yb_19', 'yb_20', 'yb_21', 'yb_22', 'yb_23',
       'household']]

In [19]:
pd.set_option('display.max_columns', 500)
final_combined_df.head()

,date,ya_0,ya_1,ya_2,ya_3,ya_4,ya_5,ya_6,ya_7,ya_8,ya_9,ya_10,ya_11,ya_12,ya_13,ya_14,ya_15,ya_16,ya_17,ya_18,ya_19,ya_20,ya_21,ya_22,ya_23,yb_0,yb_1,yb_2,yb_3,yb_4,yb_5,yb_6,yb_7,yb_8,yb_9,yb_10,yb_11,yb_12,yb_13,yb_14,yb_15,yb_16,yb_17,yb_18,yb_19,yb_20,yb_21,yb_22,yb_23,household
0,2019-01-01,NaN,258.623868,219.704170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,751.341178,NaN,NaN,NaN,NaN,1469.909821,NaN,1098.064711,1724.732556,NaN,NaN,NaN,826.489743,799.064586,655.410763,361.739778,353.541245,457.969822,373.327867,397.389734,428.594011,449.354777,548.793967,433.749401,485.933822,523.325278,533.719757,585.885410,1145.520611,1278.981621,687.871134,635.505304,675.393873,607.006912,667.906901,883.808511,SFH10
1,2019-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,149.108422,191.580600,366.615777,270.086978,606.604199,1504.291818,843.867133,630.696224,353.070955,385.870799,288.108600,448.902512,404.847334,411.934733,489.618933,182.064111,197.513556,909.217434,800.747231,553.882554,359.861722,338.943245,450.440045,431.531445,433.061301,384.368011,400.904156,439.642634,644.664201,644.722101,532.810999,925.792521,536.858800,867.671589,792.443170,1188.946257,1050.928166,1191.879457,711.292833,808.901385,492.270366,SFH10
2,2019-01-03,145.748978,163.525333,226.149044,201.983711,543.307579,NaN,274.448956,NaN,NaN,326.258799,415.855866,369.822511,750.459356,961.654268,811.846267,190.941222,438.684089,528.361710,389.644022,592.646155,403.165534,467.790711,234.986422,116.480244,518.288777,468.490090,421.932208,419.282022,362.024789,517.880267,386.609255,346.134422,364.867778,450.966059,672.452355,562.625590,481.056478,500.266156,615.716341,460.239430,646.382534,1158.986665,638.477901,693.508393,647.032660,662.172935,899.004444,633.310082,SFH10
3,2019-01-04,255.325156,176.847378,184.346200,140.751845,124.090267,209.358578,266.956645,623.385441,1469.786666,706.686709,741.229623,938.705468,478.143110,239.225267,228.493845,2160.622310,2833.751402,1332.293935,1019.141622,562.583333,1419.830268,622.953754,479.983977,302.765823,541.188289,522.967126,397.111178,386.849400,369.032244,481.644166,432.814511,379.897789,373.754193,433.916045,719.502623,696.330385,501.350814,553.276089,507.981200,556.601008,751.320251,1245.592726,691.497563,663.494822,650.762378,788.039912,719.085340,782.697074,SFH10
4,2019-01-05,184.656933,141.606488,185.821444,141.884689,258.689623,204.359133,120.200822,179.790378,973.634401,1027.132490,1457.095599,1640.525335,1553.097307,1650.000492,1099.136291,1217.742687,761.470555,420.153756,355.684622,372.935844,436.748466,411.924156,351.244333,295.922644,876.004052,507.004104,495.996326,397.420244,363.212704,351.400030,415.326696,362.464778,449.898726,538.435260,642.822378,1032.368709,804.605904,669.370208,719.422082,744.053489,712.233319,1167.841445,836.863142,737.265156,666.164386,694.065156,851.967979,791.345155,SFH10


In [28]:
final_combined_df['household'] = final_combined_df['household'].apply(lambda x: int(x.split('H')[1]))